Crawl the page of the hot 100 songs from: http://www.metrolyrics.com/

In [171]:
import requests
import re
from lxml import html
import matplotlib
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
import time
import boto3
from bs4 import BeautifulSoup
import urllib3

Getting Robots.txt

In [172]:
url = "http://www.metrolyrics.com/robots.txt"
robotstxt = requests.get(url)

import pprint 
pp = pprint.PrettyPrinter(indent=4)
print(robotstxt.text)

User-agent: *
Allow: /api/v1/sitemapxml/*
Disallow: /tools/
Disallow: /api/
Disallow: /8264/
Disallow: /search.html
Disallow: /*-pictures.html$
Allow: /*-lyrics-*-pictures.html$

User-agent: ccbot
Disallow: /

User-Agent: msnbot
Crawl-delay: 2

User-Agent: bingbot
Crawl-delay: 2

User-Agent: Yandex
Crawl-delay: 2



# Capturing the 'title', 'artist', 'link', and 'genre' of each genre's Top 100 Songs

In [204]:
def get_song_lyrics(link=str):
    response = requests.get(link, 'lxml')
    soup = BeautifulSoup(response.text, 'lxml')
    return soup


def clean_song_lyrics(link=str):
    lyrics_soup = get_song_lyrics(link)
    clean_lyrics = []
    line_count=0
    # output counter
    print('\n\n STARTED CLEANING LYRICS FOR SONG: \n\n ', link, '\n\n ')
    for text in lyrics_soup.find_all('p', class_='verse'):
        clean_lyrics.append(text.text)
        line_count+=1
    print('\n\nSONG CONTAINS  ' + str(line_count) + '  VERSES \n\n ')
    print('\n\n FINISHED CLEANING SONG: \n\n ', link, ' \n\n ')
    print('\n\n HERE ARE CLEANED LYRICS: \n\n ', clean_lyrics, ' \n\n ')
    return clean_lyrics

BeautifulSoup Web Crawler

In [205]:
top_100_urls = ['http://www.metrolyrics.com/top100-hiphop.html', 
                'https://www.metrolyrics.com/top100-rb.html', 
                'http://www.metrolyrics.com/top100-rock.html',
                'https://www.metrolyrics.com/top100-country.html',
                'https://www.metrolyrics.com/top100-pop.html']
song_links = []

for i in range(len(top_100_urls)):
    source_code = requests.get(top_100_urls[i])
    soup = BeautifulSoup(source_code.text, 'lxml')

    #Delete Counter after done testing
    counter = soup.find('div', class_='content').find_all('li')
    # for loop for getting the #1-#20 ranked songs from the site
    
    for link in soup.find_all('span', class_='song'):
        href = link.find('a', class_='song-link').get('href')
        song_title = link.find('a', class_='song-link').text
        song_lyrics = clean_song_lyrics(href)
        artist = link.find('a', class_='subtitle').get('title')
        genre = re.split('(?<=top100-)(.*?)(?=.html)', str(top_100_urls[i]))[1]
        song_links.append({'link': href, 'title': song_title[:-7], 'lyrics': song_lyrics,
                          'artist': artist, 'genre': genre})

    # for loop for getting the #21-#100 ranked songs from the site

    for link in soup.find('div', class_='song-list').find_all('li'):
        href = link.find('a', class_='title').get('href')
        song_title = link.find('a', class_='title').string
        song_lyrics = clean_song_lyrics(href)
        artist = link.find('a', class_='subtitle').get('title')
        genre = re.split('(?<=top100-)(.*?)(?=.html)', str(top_100_urls[i]))[1]
        song_links.append({'link': href, 'title': song_title, 'lyrics': song_lyrics,
                          'artist': artist, 'genre': genre})

g\nI've come to give this song\nI've come to give this song\nI've come to give this song", 'Chandra Brambra Chandra Chandra Bendram...', "I've come to give this song\nFrom the world of dreams\nI've come to give this song\nFrom crystal tears\nI've come to give this song\nFor love\nI've come to give this song\nChandra Brambra Chandra Chandra Bendram."]  

 


 STARTED CLEANING LYRICS FOR SONG: 

  http://www.metrolyrics.com/tale-as-old-as-time-lyrics-disney.html 

 


SONG CONTAINS  8  VERSES 

 


 FINISHED CLEANING SONG: 

  http://www.metrolyrics.com/tale-as-old-as-time-lyrics-disney.html  

 


 HERE ARE CLEANED LYRICS: 

  ['[Intro]\nTale as old as time\nTrue as it can be\nBarely even friends\nThen somebody bends\nUnexpectedly', '[Verse 1]\nJust a little change\nSmall to say the least\nBoth a little scared\nNeither one prepared\nBeauty and the beast', '[Pre-Chorus]\nEver just the same\nEver a surprise\nEver as before\nEver just as sure\nAs the sun will rise', '[Chorus]\nTale as old 

In [177]:
len(song_links)

500

In [178]:
song_links

 Free Lyrics\t',
  'lyrics': [<generator object Tag._all_strings at 0x7fa968041190>,
   <generator object Tag._all_strings at 0x7fa968523040>,
   <generator object Tag._all_strings at 0x7fa9685230b0>,
   <generator object Tag._all_strings at 0x7fa968523120>],
  'artist': 'Vidya Vox',
  'genre': 'pop'},
 {'link': 'http://www.metrolyrics.com/deeper-lyrics-holly-rey.html',
  'title': '\nDeeper Lyrics\t',
  'lyrics': [<generator object Tag._all_strings at 0x7fa968523270>,
   <generator object Tag._all_strings at 0x7fa96859c7b0>,
   <generator object Tag._all_strings at 0x7fa96859c820>,
   <generator object Tag._all_strings at 0x7fa96859c890>,
   <generator object Tag._all_strings at 0x7fa96859c900>,
   <generator object Tag._all_strings at 0x7fa96859c970>],
  'artist': 'Holly Rey',
  'genre': 'pop'},
 {'link': 'http://www.metrolyrics.com/my-heart-will-go-on-titanic-lyrics-celine-dion.html',
  'title': '\nMy Heart Will Go On Lyrics\t',
  'lyrics': [<generator object Tag._all_strings at 0x7f

# Setting up DynamoDB Table 

In [ ]:
import boto3

In [ ]:
__TableName__ = "Songs"

In [ ]:
dynamodb = boto3.resource('dynamodb', region_name='us-east-2')
table = dynamodb.create_table(
    TableName=__TableName__,
    KeySchema=[
        {
            'AttributeName': 'ID',
            'KeyType': 'HASH'  # Partition key
        },
        {
            "AttributeName": "genre",
            "KeyType": "RANGE"
        }
    ],
    AttributeDefinitions=[
        {
            'AttributeName': 'ID',
            'AttributeType': 'N'
        },
        {
            'AttributeName': 'genre',
            'AttributeType': 'S'
        }

    ],
    ProvisionedThroughput={
        'ReadCapacityUnits': 10,
        'WriteCapacityUnits': 10
    }
)
print("Table status:", table.table_status)

In [ ]:
client = boto3.client('dynamodb', 'us-east-2')
DB = boto3.resource('dynamodb', 'us-east-2')
table = DB.Table(__TableName__)

Method 'crawl_web' for crawling the song_links list and putting the data into the database

In [ ]:
def add_songs_to_db(to_crawl):
    for indx, val in enumerate(to_crawl):  # OR: while to_crawl:
        response = requests.get(to_crawl[indx]['link'], 'html.lxml')
        table.put_item(Item={
            'ID': indx,
            'genre': to_crawl[indx]['genre'],
            'title': to_crawl[indx]['title'],
            'artist': to_crawl[indx]['artist'],
            'url': to_crawl[indx]['link'],
        })
        crawler_sleep_timer = (np.random.randint(8, size=32) + 5)

In [ ]:
add_songs_to_db(song_links)

In [ ]:
table.get_item()

Method 'clean_song_lyrics' for cleaning song lyrics into appropriate format for storage

## WILL LIKELY NEED TO WRITE A METHOD FOR UPDATING LYRICS IN TABLE

In [ ]:
def clean_song_lyrics(response):
    cleaned_lyrics = []
    soup = BeautifulSoup(response.text, 'lxml.')
    for text in soup.find_all('p', {'class' : 'verse'}):
        cleaned_lyrics.append(text)
    return cleaned_lyrics

In [ ]:
# for loop for updating songs with cleaned song lyrics

for song in song_links

In [ ]:
clean_lyrics = clean_song_lyrics(response)

In [ ]:
print(clean_lyrics)

In [ ]:
type(clean_lyrics)

retrieves the text of the lyrics element

In [ ]:
clean_lyrics = [] 

for url in all_lyrics:
    clean_lyric = url
    clean_lyrics.append(url.text_content())
    
all_lyrics.extend(clean_lyrics)

"list does not have text content error" means that your're referenceing a list, reference the loop

In [ ]:
all_lyrics[0]

In [ ]:
len(clean_lyrics)

In [ ]:
print(clean_lyrics)